# Otimização do modelo ESN para o subsistema N

## Importar bibliotecas

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import pandas as pd
import numpy as np

import random
from itertools import product

import pickle

from torchesn.nn import ESN
from torchesn import utils

In [2]:
subsistema = 'N'

## Importando dados

In [3]:
dados_treinamento = pd.read_csv(f'../../Preprocessamento/{subsistema}_treinamento.csv', index_col='DataHora')
dados_treinamento.index = pd.to_datetime(dados_treinamento.index, format="%Y-%m-%d %H:%M:%S")
dados_treinamento

,Carga,Temperatura,seg,ter,qua,qui,sex,sab,dom,seno_dia_semana,...,mai,jun,jul,ago,set,out,nov,dez,seno_mes,cosseno_mes
DataHora,,,,,,,,,,,,,,,,,,,,,
2018-01-03 00:00:00,-0.463425,-0.226717,0,0,1,0,0,0,0,0.974928,...,0,0,0,0,0,0,0,0,5.000000e-01,0.866025
2018-01-03 01:00:00,-0.473038,-0.226717,0,0,1,0,0,0,0,0.974928,...,0,0,0,0,0,0,0,0,5.000000e-01,0.866025
2018-01-03 02:00:00,-0.511504,-0.258305,0,0,1,0,0,0,0,0.974928,...,0,0,0,0,0,0,0,0,5.000000e-01,0.866025
2018-01-03 03:00:00,-0.536532,-0.287854,0,0,1,0,0,0,0,0.974928,...,0,0,0,0,0,0,0,0,5.000000e-01,0.866025
2018-01-03 04:00:00,-0.560466,-0.301101,0,0,1,0,0,0,0,0.974928,...,0,0,0,0,0,0,0,0,5.000000e-01,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 19:00:00,-0.307824,-0.153352,0,0,0,0,0,0,1,-0.781831,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.000000
2023-12-31 20:00:00,-0.199531,-0.151314,0,0,0,0,0,0,1,-0.781831,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.000000
2023-12-31 21:00:00,-0.184359,-0.196148,0,0,0,0,0,0,1,-0.781831,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.000000


In [4]:
dados_treinamento_2019 = dados_treinamento[dados_treinamento.index.year != 2018]
dados_treinamento_2020 = dados_treinamento_2019[dados_treinamento_2019.index.year != 2019]
dados_treinamento_2021 = dados_treinamento_2020[dados_treinamento_2020.index.year != 2020]
dados_treinamento_2022 = dados_treinamento_2021[dados_treinamento_2021.index.year != 2021]
dados_treinamento_2023 = dados_treinamento_2022[dados_treinamento_2022.index.year != 2022]

In [5]:
dados_validacao = pd.read_csv(f'../../Preprocessamento/{subsistema}_validacao.csv', index_col='DataHora')
dados_validacao.index = pd.to_datetime(dados_validacao.index, format="%Y-%m-%d %H:%M:%S")
dados_validacao

,Carga,Temperatura,seg,ter,qua,qui,sex,sab,dom,seno_dia_semana,...,mai,jun,jul,ago,set,out,nov,dez,seno_mes,cosseno_mes
DataHora,,,,,,,,,,,,,,,,,,,,,
2023-12-30 00:00:00,-0.000783,-0.185959,0,0,0,0,0,1,0,-0.974928,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.0
2023-12-30 01:00:00,-0.011805,-0.170675,0,0,0,0,0,1,0,-0.974928,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.0
2023-12-30 02:00:00,-0.026000,-0.183921,0,0,0,0,0,1,0,-0.974928,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.0
2023-12-30 03:00:00,-0.060026,-0.183921,0,0,0,0,0,1,0,-0.974928,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.0
2023-12-30 04:00:00,-0.094685,-0.183921,0,0,0,0,0,1,0,-0.974928,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,0.071314,-0.045343,0,1,0,0,0,0,0,0.781831,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.0
2024-12-31 20:00:00,0.050805,-0.106481,0,1,0,0,0,0,0,0.781831,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.0
2024-12-31 21:00:00,0.090677,-0.138068,0,1,0,0,0,0,0,0.781831,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.0


## Criação dos Datasets

In [ ]:
class Dados(Dataset):
    def __init__(self, dados, modelo, entrada_carga = 48, horas_de_previsao = 24, horizonte = 2):
        # Recebendo e tratando os dados
        self.dados = dados

        # Pega dados de carga
        self.carga = self.dados['Carga']

        # Pega dados de temperatura
        self.temperatura = self.dados['Temperatura']
        
        # Pega dados de dia da semana no formato sen cos
        self.lista_dias_semana_S = list(self.dados.columns[2:4])
        self.dias_semana_S = self.dados[self.lista_dias_semana_S]

        # Pega dados de mes no formato sen cos
        self.lista_mes_S = list(self.dados.columns[4:])
        self.meses_S = self.dados[self.lista_mes_S]

        # Configurando outras variaveis
        self.entrada_carga = 48
        self.horas_de_previsao = horas_de_previsao
        self.horizonte = horizonte

        # Separa os dados
        self.entrada = []
        self.previsao = []
        for idx in range((len(self.carga) // 24) - self.entrada_carga // 24 - self.horizonte + 1):
            # Pula dia a dia
            idx = idx * 24
            
            # Salva o historico de carga
            carga_hist = self.carga[idx: idx + self.entrada_carga]

            # Salva os dados de temperatura do dia da previsão
            temperatura_H = self.temperatura[idx + self.entrada_carga + (self.horizonte - 1) * 24 : idx + self.entrada_carga + (self.horizonte) * 24]

            # Salva os dados de temperatura mínima, media e máxima
            temperatura_P = pd.Series([temperatura_H.min(), temperatura_H.mean(), temperatura_H.max()])
            
            # Salva os dados de dia da semana no formato sen cos
            dia_S = self.dias_semana_S.iloc[idx + self.entrada_carga + (self.horizonte - 1) * 24]

            # Salva os dados de mes no formato sen cos
            mes_S = self.meses_S.iloc[idx + self.entrada_carga + (self.horizonte - 1) * 24]

            # Gera as entradas concatenando os dados
            if modelo == 'M1_1':
                previsor_concat = pd.concat([carga_hist]).values
            elif modelo == 'M2_1':
                previsor_concat = pd.concat([carga_hist, temperatura_H]).values
            elif modelo == 'M2_2':
                previsor_concat = pd.concat([carga_hist, temperatura_P]).values
            elif modelo == 'M2_3':
                previsor_concat = pd.concat([carga_hist, dia_S]).values
            elif modelo == 'M2_4':
                previsor_concat = pd.concat([carga_hist, mes_S]).values
            elif modelo == 'M3_1':
                previsor_concat = pd.concat([carga_hist, temperatura_H, dia_S]).values
            elif modelo == 'M3_2':
                previsor_concat = pd.concat([carga_hist, temperatura_H, mes_S]).values
            elif modelo == 'M3_3':
                previsor_concat = pd.concat([carga_hist, temperatura_P, dia_S]).values
            elif modelo == 'M3_4':
                previsor_concat = pd.concat([carga_hist, temperatura_P, mes_S]).values
            elif modelo == 'M3_5':
                previsor_concat = pd.concat([carga_hist, dia_S, mes_S]).values
            elif modelo == 'M4_1':
                previsor_concat = pd.concat([carga_hist, temperatura_H, dia_S, mes_S]).values
            elif modelo == 'M4_2':
                previsor_concat = pd.concat([carga_hist, temperatura_P, dia_S, mes_S]).values
                
            self.entrada.append(previsor_concat)

            # Gera os dados de saida
            saida_carga = self.carga[idx + self.entrada_carga + (self.horizonte - 1) * 24 : idx + self.entrada_carga + (self.horizonte) * 24]
            self.previsao.append(saida_carga)
            
        self.entrada = torch.tensor(np.array(self.entrada), dtype = torch.float32)
        self.previsao = torch.tensor(np.array(self.previsao), dtype = torch.float32)

    def __len__(self):
        return (len(self.carga) // 24) - self.entrada_carga // 24 - self.horizonte + 1

    def __getitem__(self, idx):
        return self.entrada[idx], self.previsao[idx]

## Função perda

In [ ]:
def treinar(params):
    # Variaveis otimizadas
    modelo_entrada, n_neuronios, ano_entrada, otm, washout, r_espectral, t_vazamento = params
    lista_modelo_entrada = (
        [f"M1_{i}" for i in range(1, 2)] +
        [f"M2_{i}" for i in range(1, 5)] +
        [f"M3_{i}" for i in range(1, 6)] + 
        [f"M4_{i}" for i in range(1, 3)]
    )
    lista_modelo_n_entradas = [
        48,
        48+24, 48+3, 48+2, 48+2,
        48+24+2, 48+24+2,
        48+3+2, 48+3+2,
        48+2+2,
        48+24+2+2, 48+3+2+2
    ]
    n_entradas = lista_modelo_n_entradas[modelo_entrada]
    modelo_entrada = lista_modelo_entrada[modelo_entrada]
    lista_n_neuronios = [
        24,
        round(0.25*(2*n_entradas-24)+24),
        round(0.5*(2*n_entradas-24)+24),
        round(0.75*(2*n_entradas-24)+24),
        2*n_entradas
    ]
    n_neuronios = lista_n_neuronios[n_neuronios]
    lista_ano_entrada = [2018, 2020, 2022]
    ano_entrada = lista_ano_entrada[ano_entrada]

    lista_otm = ['svd', 'inv']
    otm = lista_otm[otm]

    lista_washout = [0.05, 0.1, 0.2]
    washout = lista_washout[washout]

    lista_r_espectral = [0.8, 0.9, 0.99]
    r_espectral = lista_r_espectral[r_espectral]

    lista_t_vazamento = [0.05, 0.1, 0.3]
    t_vazamento = lista_t_vazamento[t_vazamento]

    # Configurações
    num_seeds = 10 # Modificar
    lista_melhor_loss = []
    criterio = nn.MSELoss()

    # Loop de treinamento
    for seed in range(1, num_seeds + 1):

        if ano_entrada == 2018:
            treinamento_dataset = Dados(dados_treinamento, modelo_entrada)
        elif ano_entrada == 2020:
            treinamento_dataset = Dados(dados_treinamento_2020, modelo_entrada)
        else:
            treinamento_dataset = Dados(dados_treinamento_2022, modelo_entrada)

        validacao_dataset = Dados(dados_validacao, modelo_entrada)

        treinamento_dataloader = DataLoader(
            treinamento_dataset,
            batch_size=len(treinamento_dataset),
            shuffle=False
        )

        validacao_dataloader = DataLoader(
            validacao_dataset,
            batch_size=len(validacao_dataset),
            shuffle=False
        )

        # MODELO ESN
        rede = ESN(
            input_size=n_entradas,
            hidden_size=n_neuronios,
            output_size=24,
            readout_training = otm,
            nonlinearity = 'tanh',
            spectral_radius = r_espectral,
            leaking_rate = t_vazamento
        )

        # TREINAMENTO
        for entrada_b, saida_b in treinamento_dataloader:
            entrada_treinamento = entrada_b.unsqueeze(1)  
            saida_treinamento = saida_b.unsqueeze(1)    

        w = [int(washout * len(treinamento_dataset))]

        y_flat = utils.prepare_target(
            saida_treinamento,
            [len(treinamento_dataset)],
            w
        )

        # Acumula estatísticas do readout
        rede(entrada_treinamento, w, None, y_flat)
        rede.fit()
        output, hidden = rede(entrada_treinamento, w)

        # VALIDAÇÃO
        rede.eval()
        perdas_validacao = []

        with torch.no_grad():
            for entrada_b, saida_b in validacao_dataloader:
                entrada_validacao = entrada_b.unsqueeze(1)  
                saida_validacao = saida_b.unsqueeze(1) 

                previsao, _ = rede(entrada_validacao, [0])

                previsao = previsao.squeeze(0) 
                saida_validacao = saida_validacao.squeeze(0)
                perda = criterio(previsao, saida_validacao)
                perdas_validacao.append(perda.item())

        lista_melhor_loss.append(np.mean(perdas_validacao))

    # Retorno para o AG
    return np.mean(lista_melhor_loss)


## Otimização

In [9]:
class AG:
    def __init__(self, limites, tamanho_populacao, geracoes, taxa_mutacao, funcao_custo):
        self.limites = limites
        self.numero_variaveis = len(self.limites)
        self.tamanho_populacao = tamanho_populacao
        self.geracoes = geracoes
        self.taxa_mutacao = taxa_mutacao
        self.funcao_custo = funcao_custo
        self.geracao_atual = 0

        intervalos = [range(a, b+1) for a, b in limites]
        possibilidades = [comb for comb in product(*intervalos)]
        self.dic_historico = {(possibilidade): [] for possibilidade in possibilidades}

        self.populacao = []

    def criar_individuo(self):
        return [round(random.uniform(a - 0.49, b + 0.49)) for (a, b) in self.limites]

    def criar_populacao(self):
        return [self.criar_individuo() for _ in range(self.tamanho_populacao)]

    def avaliar_fitness(self):
        custos = []
        for ind in self.populacao:
            chave = tuple(ind)
            if self.dic_historico[chave]:
                print(f'Indivíduo já avaliado.')
                custos.append(self.dic_historico[chave][0])
            else:
                fit = self.funcao_custo(ind)
                custos.append(fit)
                self.dic_historico[chave].append(fit)
            print(f'Indivíduo: {ind}. Fit: {custos[-1]}')
        self.fitness = [1 / (1 + c) for c in custos]

    def selecao_roleta(self):
        soma = sum(self.fitness)
        pick = random.uniform(0, soma)
        atual = 0
        for ind, fit in zip(self.populacao, self.fitness):
            atual += fit
            if atual > pick:
                return ind
        return self.populacao[-1]

    def cruzamento(self, pai1, pai2):
        n = len(pai1)
        indices_pai1 = random.sample(range(n), k=n//2)
        filho = [pai1[i] if i in indices_pai1 else pai2[i] for i in range(n)]
        return filho

    def mutacao(self, ind):
        for i in range(self.numero_variaveis):
            if random.random() < self.taxa_mutacao:
                a, b = self.limites[i]
                ind[i] = round(random.uniform(a - 0.49, b + 0.49))
        return ind

    def salvar(self, nome_arquivo='ag_estado.pkl'):
        with open(nome_arquivo, 'wb') as f:
            pickle.dump(self, f)
        print(f"Estado salvo em '{nome_arquivo}'.")

    @staticmethod
    def carregar(nome_arquivo='ag_estado.pkl'):
        with open(nome_arquivo, 'rb') as f:
            ag = pickle.load(f)
        print(f"Estado carregado de '{nome_arquivo}'.")
        return ag

    def algoritmo_genetico(self, continuar=False):
        if not continuar or not self.populacao:
            self.populacao = self.criar_populacao()

        for geracao in range(self.geracao_atual, self.geracoes):
            print(f'\nGeração {geracao + 1}')
            self.avaliar_fitness()

            # Elitismo de 2 melhores
            indices_melhores = sorted(range(len(self.fitness)), key=lambda i: self.fitness[i], reverse=True)[:2]
            elite = [self.populacao[i] for i in indices_melhores]
            nova_pop = elite[:]

            while len(nova_pop) < self.tamanho_populacao:
                pai1 = self.selecao_roleta()
                pai2 = self.selecao_roleta()
                filho = self.cruzamento(pai1, pai2)
                filho = self.mutacao(filho)
                nova_pop.append(filho)

            self.populacao = nova_pop
            print(f"Elite: {elite}")

            # Salvar estado atual
            self.salvar()
            self.geracao_atual += 1

## Rodada

In [ ]:
busca_entrada = AG(
    limites = [(0, 11), (0, 4), (0, 2), (0, 1), (0, 2), (0, 2), (0, 2)], 
    tamanho_populacao = 70, 
    geracoes = 10, 
    taxa_mutacao = 0.03, 
    funcao_custo = treinar)
# Variaveis para otimizar:
# (Modelo de entrada), (Tamanho do reservatorio), (ano inicial da entrada), (washout), (raio espectral), (taxa de vazamento) 
# 12 entradas, entre o tamanho da entrada até 2x o mesmo valor, 2018-2020-2022, svd-inv, 5%-10%-20%, 0.8-0.9-0.99, 0.05-0.1-0.3

In [11]:
busca_entrada.algoritmo_genetico()


Geração 1
Indivíduo: [1, 2, 1, 1, 0, 2, 0]. Fit: 1.2215957343578339
Indivíduo: [4, 3, 1, 1, 2, 2, 1]. Fit: 0.7830563515424729
Indivíduo: [6, 0, 0, 0, 1, 2, 0]. Fit: 0.22221729904413223
Indivíduo: [5, 1, 0, 1, 2, 1, 1]. Fit: 0.12091220803558826
Indivíduo: [4, 0, 0, 0, 2, 2, 2]. Fit: 0.021631100215017796
Indivíduo: [1, 1, 0, 1, 0, 2, 1]. Fit: 0.5839594230055809
Indivíduo: [0, 4, 0, 0, 0, 2, 2]. Fit: 0.2655031569302082
Indivíduo: [6, 4, 1, 0, 1, 0, 1]. Fit: 1.5096264705061913
Indivíduo: [5, 1, 0, 1, 0, 0, 0]. Fit: 0.36974995881319045
Indivíduo: [2, 2, 0, 0, 1, 1, 1]. Fit: 2.2647356301546098
Indivíduo: [3, 0, 0, 0, 2, 2, 2]. Fit: 0.03409914346411824
Indivíduo: [6, 0, 1, 0, 0, 1, 1]. Fit: 0.09060378102585673
Indivíduo: [11, 4, 0, 1, 1, 0, 0]. Fit: 2.585858500003815
Indivíduo: [7, 1, 2, 1, 2, 2, 2]. Fit: 0.016889741457998753
Indivíduo: [0, 4, 2, 1, 2, 2, 1]. Fit: 1.555503249168396
Indivíduo: [2, 4, 0, 1, 2, 2, 0]. Fit: 2.324045905470848
Indivíduo: [11, 3, 1, 0, 0, 1, 0]. Fit: 0.823080039024

In [12]:
modelo_entrada, n_neuronios, ano_entrada, otm, washout, r_espectral, t_vazamento = busca_entrada.populacao[0]
lista_modelo_entrada = (
    [f"M1_{i}" for i in range(1, 2)] +
    [f"M2_{i}" for i in range(1, 5)] +
    [f"M3_{i}" for i in range(1, 6)] + 
    [f"M4_{i}" for i in range(1, 3)]
)
lista_modelo_n_entradas = [
    48,
    48+24, 48+3, 48+2, 48+2,
    48+24+2, 48+24+2,
    48+3+2, 48+3+2,
    48+2+2,
    48+24+2+2, 48+3+2+2
]
n_entradas = lista_modelo_n_entradas[modelo_entrada]
modelo_entrada = lista_modelo_entrada[modelo_entrada]
lista_n_neuronios = [
    24,
    round(0.25*(2*n_entradas-24)+24),
    round(0.5*(2*n_entradas-24)+24),
    round(0.75*(2*n_entradas-24)+24),
    2*n_entradas
]
n_neuronios = lista_n_neuronios[n_neuronios]
lista_ano_entrada = [2018, 2020, 2022]
ano_entrada = lista_ano_entrada[ano_entrada]

lista_otm = ['svd', 'inv']
otm = lista_otm[otm]

lista_washout = [0.05, 0.1, 0.2]
washout = lista_washout[washout]

lista_r_espectral = [0.8, 0.9, 0.99]
r_espectral = lista_r_espectral[r_espectral]

lista_t_vazamento = [0.05, 0.1, 0.3]
t_vazamento = lista_t_vazamento[t_vazamento]
print(f'O melhor modelo é: {modelo_entrada}, {n_neuronios}, {ano_entrada}, {otm}, {washout}, {r_espectral}, {t_vazamento}')

O melhor modelo é: M3_1, 24, 2022, inv, 0.1, 0.8, 0.3
